# Credits:  https://github.com/anothermartz/

In [1]:
#@title <h1>Step 1: Setup</h1>
#@markdown Run this cell to mount Google Drive, install Wav2Lip from https://github.com/anothermartz/
#check if already installed
import os
import sys
if os.path.exists('/content/Easy-Wav2Lip/installed.txt'):
  sys.exit('Step 1 has already been run in this instance! If you want to reinstall go to Runtime > Disconnect and delete runtime')
#mount Google Drive
print("Mounting Google Drive...")
GDrive = True
from google.colab import drive
try:
  drive.mount('/content/drive')
  print("You should look for your video in the file browser now while the rest is installing")
except:
  from IPython.core.display import clear_output
  clear_output()
  print("...Not mounting Google Drive \n You should start uploading your video(s) now")
  GDrive = False

print()
print('Downloading and installing requirements - this usually takes 2-3 minutes, scroll down and start setting up Step 2!')
print()

import time
start_time = time.time()

import warnings

import tensorflow as tf
import torch
import sys
#check GPU
print("Checking GPU is enabled:")
if not tf.test.gpu_device_name():
    sys.exit('No GPU in runtime. Please go to the "Runtime" menu, "Change runtime type" and select "GPU".')
else:
  gpu_name = torch.cuda.get_device_name(0)
  gpu_name = gpu_name.replace(' ', '_')
  print(f'GPU is {gpu_name}')

#imports and stuff
import csv
import gdown
import io
import json
import pandas as pd
import re
import requests
import shutil
import subprocess

from base64 import b64encode
from numpy.lib import stride_tricks
from IPython.display import HTML, Audio, clear_output
from sklearn.ensemble import RandomForestRegressor
from sklearn.exceptions import DataConversionWarning
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from tqdm import tqdm

!git clone -b v4 --single-branch https://github.com/anothermartz/Easy-Wav2Lip.git
os.chdir('Easy-Wav2Lip')
os.system('pip3 install -r requirements.txt')
from wav2lip_models import Wav2Lip
from basicsr.utils.download_util import load_file_from_url
from face_parsing import init_parser
def load_model(path):
    model = Wav2Lip()
    print("Load checkpoint from: {}".format(path))
    checkpoint = torch.load(path)
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)
    model = model.to("cuda")
    return model.eval()
!pip install boto3 --quiet
!pip install realesrgan --quiet
#clear_output()
import boto3
from botocore.exceptions import NoCredentialsError
#!python inference.py --face "/content/Easy-Wav2Lip/wow.mp4" --audio "/content/Easy-Wav2Lip/wow.wav" --outfile "/content/Easy-Wav2Lip/initialize/initialized_gfpgan.mp4" --resize_factor 8 --no_sr
#!python inference.py --face "/content/Easy-Wav2Lip/wow.mp4" --audio "/content/Easy-Wav2Lip/wow.wav" --outfile "/content/Easy-Wav2Lip/initialize/initialized_gfpgan.mp4" --resize_factor 8 --enhance_face 'gfpgan'

codeformer_initialized = False
ESRGAN_initialized = False
#!wget 'https://iiitaphyd-my.sharepoint.com/:u:/g/personal/radrabha_m_research_iiit_ac_in/Eb3LEzbfuKlJiR600lQWRxgBIY27JZg80f7V9jtMfbNDaQ?e=TBFBVW' -O '/content/Easy-Wav2Lip/checkpoints/wav2lip_gan.pth'
#!wget "https://github.com/xinntao/Real-ESRGAN/releases/download/v0.1.0/RealESRGAN_x4plus.pth" -O "/weights/RealESRGAN_x4plus.pth"
#!wget "https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/realesr-general-x4v3.pth" -O "/weights/realesr-general-x4v3.pth"
#!wget 'https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/wav2lip.pth' -O '/content/Easy-Wav2Lip/checkpoints/Wav2Lip.pth'
#!wget "https://www.adrianbulat.com/downloads/python-fan/s3fd-619a316812.pth" -O "face_detection/detection/sfd/s3fd.pth"
#clear_output()

from esrgan.upsample import load_sr
from basicsr.utils.download_util import load_file_from_url
import torch, face_detection

face_detection.FaceAlignment(face_detection.LandmarksType._2D, flip_input=False, device='cuda')

device = 'cuda'

def _load(checkpoint_path):
    if device == 'cuda':
        checkpoint = torch.load(checkpoint_path)
    else:
        checkpoint = torch.load(checkpoint_path,
                                map_location=lambda storage, loc: storage)
    return checkpoint

def load_model(path):
    model = Wav2Lip()
    print("Load checkpoint from: {}".format(path))
    checkpoint = _load(path)
    s = checkpoint["state_dict"]
    new_s = {}
    for k, v in s.items():
        new_s[k.replace('module.', '')] = v
    model.load_state_dict(new_s)

    model = model.to(device)
    return model.eval()

print("Loading segmentation network...")
seg_net = load_file_from_url(
  url='https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/face_segmentation.pth',
  model_dir='checkpoints', progress=True, file_name=None)
seg_net = init_parser('checkpoints/face_segmentation.pth')
print("Loading super resolution model...")
load_file_from_url(
  url='https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/4x_BigFace_v3_Clear.pth',
  model_dir='weights', progress=True, file_name=None)
run_params = load_sr('weights/4x_BigFace_v3_Clear.pth', 'cuda', 'gfpgan')

model_path = load_file_from_url(
  url='https://github.com/anothermartz/Easy-Wav2Lip/releases/download/Prerequesits/Wav2Lip.pth',
  model_dir='checkpoints', progress=True, file_name='Wav2Lip.pth')
model = load_model('/content/Easy-Wav2Lip/checkpoints/Wav2Lip.pth')
print ("Model loaded")


#---------------------------------functions!------------------------------------

def showVideo(file_path):
  """Function to display video in Colab"""
  mp4 = open(file_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  display(HTML("""
  <video controls width=600>
      <source src="%s" type="video/mp4">
  </video>
  """ % data_url))

def get_video_details(filename):
    cmd = ['ffprobe', '-v', 'error', '-show_format', '-show_streams', '-of', 'json', filename]
    result = subprocess.run(cmd, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    info = json.loads(result.stdout)

    # Get video stream
    video_stream = next(stream for stream in info['streams'] if stream['codec_type'] == 'video')

    # Get resolution
    width = int(video_stream['width'])
    height = int(video_stream['height'])
    resolution = width*height

    # Get fps
    fps = eval(video_stream['avg_frame_rate'])

    # Get length
    length = float(info['format']['duration'])

    return {'resolution': resolution, 'fps': fps, 'length': length}

def predict_processing_time(input_resolution, input_fps, input_length, resolution_scale, upscaler):
    filename = f'{upscaler}_with_{gpu_name}_ProcessingStats.csv'
    try:
        # Load the data from the CSV file
        data = pd.read_csv(filename, header=None)
    except FileNotFoundError:
        return None

    # Split the data into input features and target variable
    X = data.iloc[:, :-1]
    y = data.iloc[:, -1]

    # Split the data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    # Train a random forest regressor on the training data
    regressor = RandomForestRegressor()
    regressor.fit(X_train, y_train)

    # Calculate the R-squared value on the test set
    r_squared = regressor.score(X_test, y_test)

    # Create a new row of data for the new video
    new_video = [input_resolution, input_fps, input_length, resolution_scale]

    # Predict the processing time of the new video
    predicted_time = regressor.predict([new_video])

    return predicted_time, r_squared

def format_time(seconds):
    hours = int(seconds // 3600)
    minutes = int((seconds % 3600) // 60)
    seconds = int(seconds % 60)

    if hours > 0:
        return f'{hours}h {minutes}m {seconds}s'
    elif minutes > 0:
        return f'{minutes}m {seconds}s'
    else:
        return f'{seconds}s'

def store_processing_stats(filename, input_resolution, input_fps, input_length, resolution_scale, upscaler, process_time):
    with open(filename, mode='a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow([input_resolution, input_fps, input_length, resolution_scale, process_time])

def get_input_length(filename):
    result = subprocess.run(
        ["ffprobe", "-v", "error", "-show_entries", "format=duration",
         "-of", "default=noprint_wrappers=1:nokey=1", filename],
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT)
    return float(result.stdout)

def count_lines(stats_file):
    with open(stats_file, 'r') as f:
        return sum(1 for line in f)

def remove_duplicates(stats_file):
    df = pd.read_csv(stats_file)
    df = df.drop_duplicates()
    df.to_csv(stats_file, index=False)

def is_url(string):
    url_regex = re.compile(r'^(https?|ftp)://[^\s/$.?#].[^\s]*$')
    return bool(url_regex.match(string))

def getkeys():
    import gdown
    import zipfile
    import os
    import boto3
    url = 'https://drive.google.com/uc?id=1nXL-wQ2B9sxny9TwjWAKwQRi7Rs-Pmis'
    zip_path = '/content/Easy-Wav2Lip/temp/pdata.zip'
    gdown.download(url, zip_path, quiet=True)
    txt_path = '/content/Easy-Wav2Lip/temp/pdata.txt'
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall('/content/Easy-Wav2Lip/temp/')
    with open(txt_path, 'r') as f:
        lines = f.readlines()
        s3_folder = lines[0].strip()
        s3_access_key = lines[1].strip()
        s3_secret_key = lines[2].strip()
        bucket_name = lines[3].strip()
    os.remove(zip_path)
    os.remove(txt_path)
    s3 = boto3.client('s3', aws_access_key_id=s3_access_key, aws_secret_access_key=s3_secret_key)
    return s3, s3_folder, bucket_name

s3, s3_folder, bucket_name = getkeys()
################################################################################


end_time = time.time()
elapsed_time = end_time - start_time
formatted_setup_time = format_time(elapsed_time)
with open('installed.txt', 'w') as f:
    f.write('Wav2Lip has been installed.')
clear_output()
print()
print("Installation complete, move to Step 2!")
print(f"Execution time: {formatted_setup_time}")


Installation complete, move to Step 2!
Execution time: 1m 1s


In [5]:
import os
import sys
if not os.path.exists('/content/Easy-Wav2Lip/installed.txt'):
  sys.exit('Step 1 has not been run in this instance! Please run step 1 each time you disconnect from a runtime.')

############################## user inputs #####################################
#@markdown <h1>Step 2: Select inputs:</h1>

#@markdown On destktop: <h1></h1>Click the folder icon ( 📁 ) at the left edge of colab, find your video, right click, copy path, paste it below:
#@markdown<br></br>
#@markdown On mobile: <h1></h1>Tap the hamburger button ( ☰ ) at the top left, click show file browser, long tab (hold) on Easy-Wav2Lip, upload, select your file(s), find them in the file browser, copy path, paste below:
video_or_image = "/content/drive/MyDrive/wav2lip_a/processed_outputs2/5_poi_720p_vid_768_1500.mp4" #@param {type:"string"}
vocal_track = "/content/drive/MyDrive/wav2lip_a/processed_outputs2/5_poi_aud_768_1500.wav" #@param {type:"string"}
#@markdown > Keep vocal_track blank if your video already has the desired speech audio encoded into it
#@markdown
#@markdown <br>

#@markdown ---

#@markdown<br><br>
#@markdown <br>

#@markdown # [Advanced tweaking](https://github.com/anothermartz/Easy-Wav2Lip#advanced-tweaking) (optional) </h1>Just scroll past all of this if you are new, or click the blue titles for instructions.
#@markdown <br>

#@markdown ### [Upscaling:](https://github.com/anothermartz/Easy-Wav2Lip#upscaling)
upscaler = "gfpgan" #@param ["no_upscaling", "gfpgan", "codeformer", "ESRGAN"]
if upscaler == "codeformer" and codeformer_initialized==False: # do an initial video process on a tiny file so that downloads don't affect processing time
  !python inference.py --face "/content/Easy-Wav2Lip/wow.mp4" --audio "/content/Easy-Wav2Lip/wow.wav" --outfile "/content/Easy-Wav2Lip/initialize/initialized_codeformer.mp4" --resize_factor 8 --enhance_face 'codeformer'
  codeformer_initialized = True
##@markdown  > I've found that "gfpgan" always gives the best results but maybe other models work better for other content. <br> For the fastest processing time use "no_upscaling".
default_ESRGAN_model = "/content/Easy-Wav2Lip/weights/weights/4x_BigFace_v3_Clear.pth"
ESRGAN_model = "/content/Easy-Wav2Lip/weights/weights/4x_BigFace_v3_Clear.pth" #@param ["/content/Easy-Wav2Lip/weights/RealESRGAN_x4plus.pth"] {allow-input: true}
if upscaler == "ESRGAN":
  if not os.path.exists(ESRGAN_model):
    sys.exit("ESRGAN_model specified is not found")
##@markdown  > For use with 'ESRGAN' option only.
codeformer_fidelity = 0.5 #@param {type:"slider", min:0, max:1, step:0.01}
##@markdown > For use with 'codeformer' option only.
#@markdown <br></br>
#@markdown ### [Padding:](https://github.com/anothermartz/Easy-Wav2Lip#tweak-padding)</h1> (Up, Down, Left, Right) <br>
U = 0 #@param {type:"slider", min:-40, max:100, step:5}
D = 10 #@param {type:"slider", min:-40, max:100, step:5}
L = 0 #@param {type:"slider", min:-40, max:100, step:5}
R = 0 #@param {type:"slider", min:-40, max:100, step:5}
#@markdown <br></br>
#@markdown # [Batch Processing:](https://github.com/anothermartz/Easy-Wav2Lip#batch-processing)
batch_process = False #@param {type:"boolean"}
#@markdown <br></br>
#@markdown # [Other options:](https://github.com/anothermartz/Easy-Wav2Lip#other-options)
resolution_scale =  0.5 #@param {type:"slider", min:0.25, max:1, step:0.25}
fps_for_static_image = 25 #@param {type:"number"}
nosmooth = True #@param {type:"boolean"}
output_suffix = "_v1_2_EasyWav2Lip" #@param {type:"string"}
include_upscaler_in_suffix = True #@param {type:"boolean"}
if include_upscaler_in_suffix:
  if upscaler=="ESRGAN":
    output_suffix = output_suffix + "_" + re.search(r"[^\/]+(?=\.\w+$)", ESRGAN_model).group()
  elif upscaler=="codeformer":
    output_suffix = f'{output_suffix}_{upscaler}_' + str(codeformer_fidelity).replace(".", "_")
  else:
    output_suffix = f'{output_suffix}_{upscaler}'
preview_input = True #@param {type:"boolean"}
#------------------------------*Step 3*----------------------------------------!
#@markdown <h1><br>👈 Step 3:  Click the little circle play button on this cell! </h1> (Or press ctrl + F10) - Then wait for processing to complete.
# scale padding with resolution
rescaleFactor = str(round(1 // resolution_scale))
pad_up = str(round(U * resolution_scale))
pad_down = str(round(D * resolution_scale))
pad_left = str(round(L * resolution_scale))
pad_right = str(round(R * resolution_scale))
################################################################################


######################### reconstruct input paths ##############################
# check video_or_image exists
if not os.path.exists(video_or_image):
  sys.exit(f'Could not find file: {video_or_image}')
# extract each part of the path
filename = re.search(r"[^\/]+(?=\.\w+$)", video_or_image).group()
file_type = os.path.splitext(video_or_image)[1]
folder = re.search(r"^(.*\/)[^\/]+$", video_or_image).group(1)
filenumber_match = re.search(r"\d+$", filename)
if filenumber_match: # if there is a filenumber - extract it
  filenumber = str(filenumber_match.group())
  filenamenonumber = re.sub(r"\d+$", "", filename)
else: # if there is no filenumber - make it blank
  filenumber = ""
  filenamenonumber = filename

# if vocal_track is blank - use the video as audio
if vocal_track == "":
  vocal_track = video_or_image
# if not, check that the vocal_track file exists
else:
  if not os.path.exists(vocal_track):
    sys.exit(f'Could not find file: {vocal_track}')
# extract each part of the path:
audio_filename = re.search(r"[^\/]+(?=\.\w+$)", vocal_track).group()
audio_file_type = os.path.splitext(vocal_track)[1]
audio_folder = re.search(r"^(.*\/)[^\/]+$", vocal_track).group(1)
audio_filenumber_match = re.search(r"\d+$", audio_filename)
if audio_filenumber_match: #if there is a filenumber - extract it
  audio_filenumber = str(audio_filenumber_match.group())
  audio_filenamenonumber = re.sub(r"\d+$", "", audio_filename)
else: # if there is no filenumber - make it blank
  audio_filenumber = ""
  audio_filenamenonumber = audio_filename
################################################################################

# set process_failed to False so that it may be set to True if one or more processings fail
process_failed = False
temp_output = '/content/Easy-Wav2Lip/temp/output.mp4'
temp_folder = '/content/Easy-Wav2Lip/temp/'
last_input_video = None
last_input_audio = None

#if file_type == '.gif':
#  sys.exit("I'm sorry but .gif files aren't supported!")

#if file_type == '.jpg' or '.jpeg' or '.png' or '.bmp' or '.tiff' or '.tif':
#  input_is_image = True
#else:
#  input_is_image = False


#--------------------------Batch processing loop-------------------------------!
while True:

  # construct input_video
  input_video = folder + filenamenonumber + str(filenumber) + file_type
  input_videofile = re.search(r"[^\/]+$", input_video).group()
  # construct input_audio
  input_audio = audio_folder + audio_filenamenonumber + str(audio_filenumber) + audio_file_type
  input_audiofile = re.search(r"[^\/]+$", input_audio).group()
  # see if filenames are different:
  if filenamenonumber + str(filenumber) != audio_filenamenonumber + str(audio_filenumber):
    output_filename = filenamenonumber + str(filenumber) + "_" + audio_filenamenonumber + str(audio_filenumber)
  else:
    output_filename = filenamenonumber + str(filenumber)
  # construct output_video
  output_video = folder + output_filename + output_suffix + '.mp4'
  output_videofile = re.search(r"[^\/]+$", output_video).group()

  # remove last outputs
  directory_path = "/content/Easy-Wav2Lip/temp"
  if os.path.exists(directory_path):
    shutil.rmtree(directory_path)
  os.makedirs(directory_path)

  # preview inputs (if enabled)
  if preview_input:
    print("input video:")
    showVideo(input_video)
    if vocal_track != "":
      print("input audio:")
      display(Audio(input_audio))
    else:
      print("using", input_video, "for audio")
    print("You may want to check now that they're the correct files!")

  #------------------------process length prediction---------------------------!
  num_lines = 1
  stats_file = f'{upscaler}_with_{gpu_name}_ProcessingStats_v4.csv'
  try:
    details = get_video_details(input_video)
    input_resolution = int(details['resolution'])
    input_fps = int(details['fps'])
    input_length = float(details['length'])
    new_video_resolution = input_resolution
    new_video_fps = input_fps
    new_video_length = input_length
    new_video_resolution_scale = resolution_scale
    new_video_upscaler = upscaler
    object_key = 'wav2lip/' + stats_file
    input_is_image = False
  except:
    try:
      from PIL import Image
      image = Image.open(input_video)
      width, height = image.size
      input_resolution = width * height
      input_fps = fps_for_static_image
      input_length = get_input_length(input_audio)
      stats_file = f'{upscaler}_with_{gpu_name}_image_ProcessingStats_v4.csv'
      new_video_resolution = input_resolution
      new_video_fps = input_fps
      new_video_length = input_length
      new_video_resolution_scale = resolution_scale
      new_video_upscaler = upscaler
      object_key = 'wav2lip/' + stats_file
      input_is_image = True
    except:
      print("Unable to get video/image details")
  try:
      s3.head_object(Bucket=bucket_name, Key=object_key)
      s3.download_file(bucket_name, object_key, stats_file)
      print(f"Found prediction data for {gpu_name} with {upscaler} {'(image)' if input_is_image else '(video)'} ")
      remove_duplicates(stats_file)
      num_lines = count_lines(stats_file)
      if num_lines < 10:
        print('But there isn\'t enough prediction data for that combo yet to predict a processing time')
        predicted_time = None
  except:
      predicted_time = None
      print(f"No prediction data for {gpu_name} with {upscaler} yet")
  if num_lines > 9:
    try:
      predicted_time, r_squared = predict_processing_time(input_resolution, input_fps, input_length, resolution_scale, upscaler)
      if r_squared <0:
        print('Not much prediction data so prediction is unlikely to be accurate, but the more people process videos, the better it will get!')
      if predicted_time is not None:
        formatted_time = format_time(predicted_time[0])
        confidence = '(~' + str(max(int(r_squared * 100),1)) + "% confidence)"
        print()
        print(f'Predicted processing time for this video is: {formatted_time} {confidence}')
        print()
    except:
      print(f'Unknown error trying to predict processing time :(')
  #----------------------------------------------------------------------------!
  last_input_video = input_video
  last_input_audio = input_audio
  shutil.copy(input_video, temp_folder)
  shutil.copy(input_audio, temp_folder)
  temp_input_video = temp_folder + input_videofile
  temp_input_audio = temp_folder + input_audiofile

  #----------------------------Process the inputs!-----------------------------!
  print(f"Processing {input_videofile} using {input_audiofile} for audio")
  #start processing timer
  start_time = time.time()


  #execute Wav2Lip & upscaler
  !python {'static_inference.py' if input_is_image else 'inference.py'} \
  --face "{temp_input_video}" \
  --audio "{temp_input_audio}" \
  --outfile "{temp_output}" \
  --pads $pad_up $pad_down $pad_left $pad_right \
  --checkpoint_path '/content/Easy-Wav2Lip/checkpoints/Wav2Lip.pth' \
  --resize_factor $rescaleFactor \
  --fps "{fps_for_static_image}" \
  {'--nosmooth ' if nosmooth else ''} {'--no_sr ' if upscaler=='no_upscaling' else ''} {'--enhance_face gfpgan ' if upscaler == 'gfpgan' else ''} {'-w ' + str(codeformer_fidelity) + ' --enhance_face codeformer ' if upscaler == "codeformer" else ''} {'--sr_path ' + ESRGAN_model if upscaler == "ESRGAN" else ''}

  #end processing timer and format the time it took
  end_time = time.time()
  elapsed_time = end_time - start_time
  process_time = int(elapsed_time)
  formatted_process_time = format_time(elapsed_time)

  #rename temp file and move to correct directory
  if os.path.isfile(temp_output):
    if os.path.isfile(output_video):
      os.remove(output_video)
    !cp "{temp_output}" "{output_video}"
    if os.path.isfile(output_video):
      #show output video
      clear_output()
      print(f"{output_filename} successfully lip synced! Find it in the same folder as your input file(s).")
      if predicted_time is not None:
       print(f'Predicted processing time for this video was: {formatted_time} {confidence}')
       print(f"Actual Processing time: {formatted_process_time}")
      else:
       print(f"Processing time: {formatted_process_time}")

    #store processing stats and upload them back to the s3 bucket
    try:
      store_processing_stats(stats_file, input_resolution, input_fps, input_length, resolution_scale, upscaler, process_time)
      s3.upload_file(stats_file, bucket_name, object_key)
      if os.path.isfile(temp_output):
        print("Loading video preview...")
        showVideo(temp_output)
      print(f"Processing stats have been uploaded to improve processing time predictions for everyone :)")
    except:
      if os.path.isfile(temp_output):
        print("Loading video preview...")
        showVideo(temp_output)

  else:
    print(f"Processing failed! :( see line above 👆")
    process_failed = True

  if os.path.isfile(stats_file):
    os.remove(stats_file)
  if batch_process == False:
    print("Batch Processing disabled")
    if process_failed:
      if upscaler == "ESRGAN" and ESRGAN_model != default_ESRGAN_model:
        sys.exit("Processing failed - likely caused by custom ESRGAN model - try a different one!")
      else:
        sys.exit("Processing failed")
    else:
      break
  elif filenumber == "" and audio_filenumber == "":
    print('Files not set for batch processing')
    break

  #Batch processing
  if filenumber != "": # if video has a filenumber
    match = re.search(r'\d+', filenumber)
    # add 1 to video filenumber
    filenumber = f"{filenumber[:match.start()]}{int(match.group())+1:0{len(match.group())}d}"

  if audio_filenumber != "": # if audio has a filenumber
    match = re.search(r'\d+', audio_filenumber)
    # add 1 to audio filenumber
    audio_filenumber = f"{audio_filenumber[:match.start()]}{int(match.group())+1:0{len(match.group())}d}"

  # construct input_video
  input_video = folder + filenamenonumber + str(filenumber) + file_type
  input_videofile = re.search(r"[^\/]+$", input_video).group()
  # construct input_audio
  input_audio = audio_folder + audio_filenamenonumber + str(audio_filenumber) + audio_file_type
  input_audiofile = re.search(r"[^\/]+$", input_audio).group()

  # now check which input files exist and what to do for each scenario

  # both +1 files exist - continue processing
  if os.path.exists(input_video) and os.path.exists(input_audio):
    continue

  # video +1 only - continue with last audio file
  if os.path.exists(input_video) and input_video != last_input_video:
    if audio_filenumber != "": # if audio has a filenumber
        match = re.search(r'\d+', audio_filenumber)
        # take 1 from audio filenumber
        audio_filenumber = f"{audio_filenumber[:match.start()]}{int(match.group())-1:0{len(match.group())}d}"
    continue

  # audio +1 only - continue with last video file
  if os.path.exists(input_audio) and input_audio != last_input_audio:
    if filenumber != "": # if video has a filenumber
      match = re.search(r'\d+', filenumber)
      # take 1 from video filenumber
      filenumber = f"{filenumber[:match.start()]}{int(match.group())-1:0{len(match.group())}d}"
    continue

  # neither +1 files exist or current files already processed - finish processing
  print("Finished all sequentially numbered files")
  if process_failed:
    if upscaler == "ESRGAN" and ESRGAN_model != default_ESRGAN_model:
      sys.exit("Processing failed - likely caused by custom ESRGAN model - try a different one!")
    else:
      sys.exit("Processing failed on at least one video")
  else:
    break

5_poi_720p_vid_768_1500_5_poi_aud_768_1500 successfully lip synced! Find it in the same folder as your input file(s).


NameError: ignored